## Декораторы функций с параметрами

Импортируем библиотеку math:

In [1]:
import math

Предположим, что у нас есть функция, которая вычисляет синус:

In [2]:
def sin_df(x):
    return math.sin(x)

In [3]:
g = math.pi/3

In [4]:
sin_df(g)

0.8660254037844386

И мы хотим вычислить производную этой функции с точностью 0.0001. Для этого можно определить декоратор:

In [5]:
def func_decorator(func):
    def wrapper(x, *args, **kwargs):
        dx = 0.0001
        return (func(x + dx, *args, **kwargs) - func(x, *args, **kwargs)) / dx
    return wrapper

In [6]:
@func_decorator
def sin_df(x):
    return math.sin(x)

In [7]:
sin_df(g)

0.4999566978958203

Но если потребуется изменить точность - придется менять декоратор. А передать ему ничего на данный момент невозможно.

Для того, чтобы иметь такую возможность, нужно модифицировать декоратор и добавить еще одну функцию:

In [8]:
def df_decorator(dx):
    def func_decorator(func):
        def wrapper(x, *args, **kwargs):
            return (func(x + dx, *args, **kwargs) - func(x, *args, **kwargs)) / dx
        return wrapper
    return func_decorator

Вся функция, которая имелась до этого, помещается в еще одну функцию, которая принимает параметр dx и возвращает ссылку на функцию внутри себя. Теперь к функции, которую декорируем, применяется именно она, и ей передается значение параметра dx:

In [9]:
@df_decorator(dx=0.000001)
def sin_df(x):
    return math.sin(x)

In [10]:
sin_df(g)

0.4999995669718871

Через иной синтаксис это выглядит следующим образом:

In [11]:
def sin_df(x):
    return math.sin(x)

In [12]:
f = df_decorator(dx=0.000001)
sin_df = f(sin_df)

In [13]:
sin_df(g)

0.4999995669718871

Или то же самое, но в одну строку без доп. переменных:

In [14]:
def sin_df(x):
    return math.sin(x)

In [15]:
sin_df = df_decorator(dx=0.000001)(sin_df)

In [16]:
sin_df(g)

0.4999995669718871

### Проблема потери имени и описания декорируемой функции

У каждой функции есть свое имя, которое доступно с помощью специальной переменной name:

In [17]:
def sin_df(x):
    return math.sin(x)

In [18]:
print(sin_df.__name__)

sin_df


Но при декорировании функции оно будет потеряно:

In [19]:
@df_decorator(dx=0.000001)
def sin_df(x):
    return math.sin(x)

In [20]:
print(sin_df.__name__)

wrapper


Эту проблему можно решив, явно прописав имя в теле декоратора:

In [21]:
def df_decorator(dx):
    def func_decorator(func):
        def wrapper(x, *args, **kwargs):
            return (func(x + dx, *args, **kwargs) - func(x, *args, **kwargs)) / dx
        wrapper.__name__ = func.__name__
        return wrapper
    
    return func_decorator

In [22]:
@df_decorator(dx=0.000001)
def sin_df(x):
    return math.sin(x)

И таким образом имя фунции сохранится:

In [23]:
print(sin_df.__name__)

sin_df


Также стоит сохранять и описание функции:

In [24]:
def sin_df(x):
    """Функция для вычисления производной синуса"""
    return math.sin(x)

In [25]:
def df_decorator(dx):
    def func_decorator(func):
        def wrapper(x, *args, **kwargs):
            return (func(x + dx, *args, **kwargs) - func(x, *args, **kwargs)) / dx
        wrapper.__name__ = func.__name__
        wrapper.__doc__ = func.__doc__
        return wrapper
    
    return func_decorator

In [26]:
@df_decorator(0.0001)
def sin_df(x):
    """Функция для вычисления производной синуса"""
    return math.sin(x)

In [27]:
print(sin_df.__doc__)

Функция для вычисления производной синуса


Но этот функционал (для сохранения имени и описания функции) - стандартный подход при декорировании, и стандартизирован на уровне языка

Нужно импортировать специальный декоратор:

In [28]:
from functools import wraps

И с помощью него декорировать вложенную функцию wrapper:

In [29]:
def df_decorator(dx):
    def func_decorator(func):
        @wraps(func)
        def wrapper(x, *args, **kwargs):
            return (func(x + dx, *args, **kwargs) - func(x, *args, **kwargs)) / dx
        
        return wrapper
    
    return func_decorator

In [30]:
@df_decorator(0.00001)
def sin_df(x):
    """Функция для вычисления производной синуса"""
    return math.sin(x)

In [31]:
print(sin_df.__name__)
print(sin_df.__doc__)

sin_df
Функция для вычисления производной синуса


И таким образом имя и описание функции будет сохранено благодаря декоратору wraps вложенной функции wrapper.